# Use Stripe with agents.json
This notebook demonstrates how to use agents.json to interact with the Stripe API. This simple example shows loading the agents.json file and using it to execute Stripe tasks.


In [ ]:
%pip install agentsjson
%pip install openai

### Setup API keys and references to `agents.json` files

In [8]:
STRIPE_API_KEY="<your-stripe-api-key>"
OPENAI_API_KEY="<your-openai-api-key>" 
agents_json_url = "https://raw.githubusercontent.com/wild-card-ai/agents-json/refs/heads/master/agents_json/stripe/agents.json"

### Load the agents.json file

In [2]:
from agentsjson.core.models import Flow
from agentsjson.core.models.bundle import Bundle
from agentsjson.core import load_agents_json

# load the agents.json file
data: Bundle = load_agents_json(agents_json_url)
agentsjson = data.agentsJson

### Define the Agent

In [3]:
query = "Create a new Stripe product for Tie Dye T-Shirts priced at $10, $15, and $30 for small, medium, and large sizes"

In [4]:
from agentsjson import ToolFormat, get_tool_prompt

# Format the flows data for the prompt
flows_context = get_tool_prompt(agentsjson)

# Create the system prompt
system_prompt = f"""You are an AI assistant that helps users interact with the Stripe API.
You have access to the following API flows:

{flows_context}

Analyze the user's request and use the appropriate API flows to accomplish the task.
You must give your arguments for the tool call as Structued Outputs JSON with keys `parameters` and `requestBody`"""

### Configure Auth

In [5]:
from agentsjson.core.models.auth import AuthType, BearerAuthConfig
auth = BearerAuthConfig(type=AuthType.BEARER, token=STRIPE_API_KEY)

### Run the agent

In [ ]:
from openai import OpenAI
from agentsjson import execute, get_tools
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ],
    tools=get_tools(agentsjson, format=ToolFormat.OPENAI),
    temperature=0
)

response = execute(agentsjson, response, format=ToolFormat.OPENAI, auth=auth)

response